In [1]:
#py_data_analysis environment
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from osgeo import gdal
from compute_ETo import *

#### Gridded variables of PET computation

In [ ]:
#read the data
src_path=r"D:\VUB\_data\mswx_ETo_inputs"

#read the data: All these datasets are from MSWX: https://journals.ametsoc.org/view/journals/bams/103/3/BAMS-D-21-0145.1.xml
# Except for wind speed (10m),Tmax, Tmin and RH are at 2m height
Tmax=xr.open_dataset(os.path.join(src_path,"Tmax_KE_data.nc"))
Tmin=xr.open_dataset(os.path.join(src_path,"Tmin_KE_data.nc"))
rh=xr.open_dataset(os.path.join(src_path,"rh_KE_data.nc"))
ws=xr.open_dataset(os.path.join(src_path,"Wind_KE_data.nc"))
# Rn_mswx=xr.open_dataset(os.path.join(src_path,"Rn_KE_data.nc"))

#use subset of the data from 2000
Tmax = Tmax.sel(time=slice("2000-01-01", '2023-10-26'))
Tmin = Tmin.sel(time=slice("2000-01-01", '2023-10-26'))
rh = rh.sel(time=slice("2000-01-01", '2023-10-26'))
ws = ws.sel(time=slice("2000-01-01", '2023-10-26'))

In [ ]:
#mean temperature
Tmean=(Tmax['air_temperature']+Tmin['air_temperature'])/2
Tmean=Tmean.to_dataset()
Tmean.attrs['units']='°C'

In [ ]:
#compute 2-m wind speed from 10m wind speed
windspeed_2m=compute_2m_wind_speed(ws,10)

In [ ]:
#compute slope of the saturation vapour pressure curve
delta=compute_slope_of_vapor_pressure_curve(Tmean['air_temperature'])

In [ ]:
#compute the saturation vapour pressure (es)
es=compute_saturation_vapor_pressure(Tmax['air_temperature'],Tmin['air_temperature'])
es=es.to_dataset()
es=es.rename({'air_temperature':'es'})
es.attrs['units']='kPa'

In [ ]:
#compute the actual vapour pressure (ea)
ea = es * (rh['relative_humidity']/100)
ea = ea.rename({'es':'ea'})

#compute dewpoint temperature
td = dew_point_temperature(Tmean['air_temperature'],rh['relative_humidity'])
ea_td = 0.6108 * np.exp((17.27 * td) / (td + 237.3))
ea_td.name = 'ea_td'
ea_td = ea_td.to_dataset()

In [ ]:
ea_tmin=0.6108 * np.exp((17.27 * Tmin['air_temperature']) / (Tmin['air_temperature'] + 237.3))

In [ ]:
#vapor pressure deficit (VPD)
vpd=es['es']-ea['ea']
#assign name
vpd.name='VPD'
#assign units
vpd.attrs['units']='kPa'
vpd=vpd.to_dataset()

In [ ]:
ea_td['ea_td'].sel(lat=1.0,lon=36.4,method='nearest').plot(figsize=(15,3.5), c='g')
ea['ea'].sel(lat=1.0,lon=36.4,method='nearest').plot(alpha=1, c='k')
#ea_tmin.sel(lat=1.0,lon=36.4,method='nearest').plot(alpha=0.5, c='r')

### Extra-terrestrial and net Radiation

In [ ]:
#extract julian day from date. The choice of Tmax is arbitrary. Any other variable could be used.
jday=Tmax.time.dt.dayofyear

latitude = Tmax.lat
longitude = Tmax.lon
time = Tmax.time
day_of_year = jday
# Create a DataArray with these dimensions
times,latitudes, longitudes  = xr.broadcast(time,latitude, longitude)

# Compute Ra using the broadcasted coordinates
Ra = extra_terrestrial_radiation(latitudes, day_of_year)

# Retain the original dataset dimensions
Ra = Ra.transpose('time','lat', 'lon')

Ra.name='Ra'
Ra.attrs['units']='MJ/m2/day'
Ra=Ra.to_dataset()

In [ ]:
#compute net radiation
Rn=compute_net_radiation(Ra['Ra'],Tmax['air_temperature'],Tmin['air_temperature'],ea['ea'])
Rn.name='Rn'
Rn.attrs['units']='MJ/m2/day'
Rn=Rn.to_dataset()

#### Psychrometric Map

> This map was generated from a DEM in QGIS

In [ ]:
DEM_file=r"D:\VUB\_data\DEM\psychrometric_constant.tif"

In [ ]:
#read the first file to get the dimensions
ds = gdal.Open(DEM_file)
band = ds.GetRasterBand(1)

#open as array
arr = band.ReadAsArray()

#get the size and coordinates
nlat,nlon = np.shape(arr)
b = ds.GetGeoTransform() #bbox, interval
#get the number of rows and columns and multiply by the interval, then add to the origin to get the coordinates
lon = np.arange(nlon)*b[1]+b[0]
lat = np.arange(nlat)*b[5]+b[3]

#assign the coordinates to the array
arr = xr.DataArray(arr,coords=[lat,lon],dims=['lat','lon'])

#assign nodata value
psychrometric = arr.where(arr!=band.GetNoDataValue())

#to reduce file size, convert to float32
psychrometric=psychrometric.astype('float32')
psychrometric.name='psychrometric_constant'

#set projection
psychrometric.attrs['crs'] = 'EPSG:4326'

### Calculate components of ETo

> Since ETo combines RN, wind speed and psychrometric constant and the arrays are in different resolutions,
> Compute the numerator and denominator terms of windspeed separately i.e.
>
> (900/273+T) x u2 x VPD >>> wind_const_n
> 
>  1+0.34 x u2 >>> wind_const_d

In [ ]:
#wind_const_n = ((900 / Tmean)) * windspeed_2m * vpd
T_factor=(900/(273.16+Tmean['air_temperature'])) * windspeed_2m['wind_speed'] 
T_factor.name='T_factor'
T_factor=T_factor.to_dataset()
wind_const_n=T_factor['T_factor']*vpd['VPD']
wind_const_n.name='wind_const_n'
wind_const_n=wind_const_n.to_dataset()


#denominator
wind_const_d = (0.34*windspeed_2m) + 1

In [ ]:
#export Rn and psychrometric constant
Rn.to_netcdf(r"D:\VUB\_data\mswx_ETo_inputs\Rn_ROI_10km.nc")
psychrometric.to_netcdf(r"D:\VUB\_data\mswx_ETo_inputs\psychrometric_constant_500m.nc")
wind_const_d.to_netcdf(r"D:\VUB\_data\mswx_ETo_inputs\wind_const_d_10km.nc")
wind_const_n.to_netcdf(r"D:\VUB\_data\mswx_ETo_inputs\wind_const_n_10km.nc")
delta.to_netcdf(r"D:\VUB\_data\mswx_ETo_inputs\delta_10km.nc")

In [ ]:
vpd.to_netcdf(r"D:\VUB\_data\mswx_ETo_inputs\VPD_10km.nc")

#### Further Processing using the xesmf_regidding.ipynb

>Because of problems of package compatibility in the environment (xesmf broke the env), regridding the datasets to a uniform resolution and finalizing ETo calculation will be completed in a different notebook